In [ ]:
import os
os.chdir("..")

In [ ]:
from src.data.data_process import DataReg
import polars as pl
import requests
import bambi as bmb
import geopandas as gpd
from pysal.lib import weights
from shapely import wkt
import pandas as pd
import arviz as az
import matplotlib.dates as mdates
import numpy as np
import causalpy as cp
import matplotlib.pyplot as plt
dr = DataReg()

In [ ]:
pr_zips = gpd.GeoDataFrame(dr.make_spatial_table().df())
pr_zips["geometry"] = pr_zips["geometry"].apply(wkt.loads)
pr_zips = pr_zips.set_geometry("geometry").set_crs("EPSG:4269", allow_override=True)
pr_zips = pr_zips.to_crs("EPSG:3395")
pr_zips["zipcode"] = pr_zips["zipcode"].astype(str)
pr_zips.plot()

In [ ]:
ranges = 1609.344 * 20

w_adaptive =  weights.distance.DistanceBand.from_dataframe(
    pr_zips, ranges, binary=True
)

In [ ]:
# Create full matrix version of weights
full_matrix, ids = w_adaptive.full()
# Set up figure with two subplots in a row
f, ax = plt.subplots(
    1, 2, figsize=(15, 10)
)
# Append weights for first polygon and plot on first subplot
pr_zips.assign(weight_0=full_matrix[10]).plot(
    "weight_0", cmap="plasma", ax=ax[0]
)

# Add titles
ax[0].set_title("Kernel centered on first tract")
# Remove axis
[ax_.set_axis_off() for ax_ in ax]
